In [3]:
# test - tree construction
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print

urdf_file = "./assets/kuka_iiwa.urdf"
root, links, robot = rmp_tree_from_urdf(urdf_file)

rmp_tree_print(root)

q = np.array([1,2,3,4,5,6,7])
dq = np.array([-1,-2,-3,-4,-5,-6,-7])

root.set_root_state(q,dq)
root.pushforward()


`- root
   |- proj_lbr_iiwa_link_1
   |  `- lbr_iiwa_link_1
   |- proj_lbr_iiwa_link_2
   |  `- lbr_iiwa_link_2
   |- proj_lbr_iiwa_link_3
   |  `- lbr_iiwa_link_3
   |- proj_lbr_iiwa_link_4
   |  `- lbr_iiwa_link_4
   |- proj_lbr_iiwa_link_5
   |  `- lbr_iiwa_link_5
   |- proj_lbr_iiwa_link_6
   |  `- lbr_iiwa_link_6
   `- proj_lbr_iiwa_link_7
      `- lbr_iiwa_link_7
root: pushforward
proj_lbr_iiwa_link_1: pushforward
proj_lbr_iiwa_link_1: x = [1.], dx = [-1.]
lbr_iiwa_link_1: pushforward
lbr_iiwa_link_1: x = [0.     0.     0.1575 1.     0.     0.    ], dx = [ 0.  0.  0.  0.  0. -1.]
proj_lbr_iiwa_link_2: pushforward
proj_lbr_iiwa_link_2: x = [1. 2.], dx = [-1. -2.]
lbr_iiwa_link_2: pushforward
lbr_iiwa_link_2: x = [ 0.          0.          0.36       -1.14159265 -1.         -1.57079633], dx = [ 0.          0.          0.          1.68294197 -1.08060461 -1.        ]
proj_lbr_iiwa_link_3: pushforward
proj_lbr_iiwa_link_3: x = [1. 2. 3.], dx = [-1. -2. -3.]
lbr_iiwa_link_3: pushforward

In [4]:
import numpy as np
from rmpflow.rmp.rbd_rmp import *
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

from scipy.spatial.transform import Rotation as R

ax, ay, az = math.pi/2, math.pi/3, math.pi/4

# euler zyx
rot_sv = s.RotZ(ax)*s.RotY(ay)*s.RotX(az)
rot = rot_sv.transpose()
r = R.from_matrix(np.array(rot))
print("Expected ax, ay, az = {}, {}, {}".format(ax,ay,az))
print(r.as_euler('zyx', degrees=False))


Expected ax, ay, az = 1.5707963267948966, 1.0471975511965976, 0.7853981633974483
[1.57079633 1.04719755 0.78539816]


In [1]:
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, PositionProjection

root, links = rmp_tree_from_urdf("./assets/kuka_iiwa.urdf")

# link5_pos = PositionProjection("link5_pos", links[''])
for key, value in links.items():
    print(key, " : ", value.name, ", ", value.parent.name, ", ", value.parent.param_map)


lbr_iiwa_link_1  :  lbr_iiwa_link_1 ,  proj_lbr_iiwa_link_1 ,  [1 0 0 0 0 0 0]
lbr_iiwa_link_2  :  lbr_iiwa_link_2 ,  proj_lbr_iiwa_link_2 ,  [1 1 0 0 0 0 0]
lbr_iiwa_link_3  :  lbr_iiwa_link_3 ,  proj_lbr_iiwa_link_3 ,  [1 1 1 0 0 0 0]
lbr_iiwa_link_4  :  lbr_iiwa_link_4 ,  proj_lbr_iiwa_link_4 ,  [1 1 1 1 0 0 0]
lbr_iiwa_link_5  :  lbr_iiwa_link_5 ,  proj_lbr_iiwa_link_5 ,  [1 1 1 1 1 0 0]
lbr_iiwa_link_6  :  lbr_iiwa_link_6 ,  proj_lbr_iiwa_link_6 ,  [1 1 1 1 1 1 0]
lbr_iiwa_link_7  :  lbr_iiwa_link_7 ,  proj_lbr_iiwa_link_7 ,  [1 1 1 1 1 1 1]


In [2]:
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print, node_array, ProjectionNode, PositionProjection, FrameProjection
import rmpflow.rmp.rmp_leaf as leaves

urdf_file = "./assets/kuka_iiwa.urdf"
root, links, robot = rmp_tree_from_urdf(urdf_file)

# attach frame projector and attractor
proj_node = PositionProjection("proj_f_lbr_iiwa_link_7", links["lbr_iiwa_link_7"])
target_pos = np.array([1,1,1])
atrc = leaves.GoalAttractorUni("attractor", proj_node, target_pos.transpose(), gain=10)

# tree printing
rmp_tree_print(root)

# simulate
# for i in range(0, 1000):
#     q = np.random.rand(7).reshape(-1,1)
#     dq = np.random.rand(7).reshape(-1,1)
    
#     qdd = root.solve(q, dq)

q = np.array([1,2,3,4,5,6,7])
dq = np.array([-1,-2,-3,-4,-5,-6,-7])

root.set_root_state(q,dq)
root.pushforward()
root.pullback()
root.resolve()



`- root
   |- proj_q_lbr_iiwa_link_1
   |  `- lbr_iiwa_link_1
   |- proj_q_lbr_iiwa_link_2
   |  `- lbr_iiwa_link_2
   |- proj_q_lbr_iiwa_link_3
   |  `- lbr_iiwa_link_3
   |- proj_q_lbr_iiwa_link_4
   |  `- lbr_iiwa_link_4
   |- proj_q_lbr_iiwa_link_5
   |  `- lbr_iiwa_link_5
   |- proj_q_lbr_iiwa_link_6
   |  `- lbr_iiwa_link_6
   `- proj_q_lbr_iiwa_link_7
      `- lbr_iiwa_link_7
         `- proj_f_lbr_iiwa_link_7
            `- attractor
root: pushforward
proj_q_lbr_iiwa_link_1: pushforward
proj_q_lbr_iiwa_link_1: x = [1.], dx = [-1.]
lbr_iiwa_link_1: pushforward
lbr_iiwa_link_1: x = [0.     0.     0.1575 1.     0.     0.    ], dx = [ 0.  0.  0.  0.  0. -1.]
proj_q_lbr_iiwa_link_2: pushforward
proj_q_lbr_iiwa_link_2: x = [1. 2.], dx = [-1. -2.]
lbr_iiwa_link_2: pushforward
lbr_iiwa_link_2: x = [ 0.          0.          0.36       -1.14159265 -1.         -1.57079633], dx = [ 0.          0.          0.          1.68294197 -1.08060461 -1.        ]
proj_q_lbr_iiwa_link_3: pushforward
p

array([  2.4124266 , -54.67338181,   2.42626953,  54.60840929,
         2.40955737,   1.24297672,   0.        ])